In [1]:
'''
※ 두 가지 가설
  1. 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
  2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
'''

'\n※ 두 가지 가설\n  1. 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.\n  2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.\n'

### 크롤링을 이용한 서울시 스타벅스 매장 목록 데이터 생성

In [8]:
#라이브러리 임포트
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
 
#webdriver 실행 후 스타벅스의 지역별 매장 검색 화면에 접속
browser = webdriver.Chrome('D:/SJY/python/datadown/chromedriver.exe')
url = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
browser.get(url)

In [9]:
#webdriver로 ‘서울’ 버튼 요소를 찾아 클릭
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(seoul_btn).click()

In [10]:
#webdriver로 ‘전체’ 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(all_btn).click()

In [11]:
#BeautifulSoup으로 HTML 파서 만들기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [12]:
#select()를 이용해 원하는 HTML 태그를 모두 찾아오기
sb_all = soup.select('ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon')
print(len(sb_all))

545


In [13]:
#첫번째 태그 문자열 살펴보기
sb_all[4]

<li class="quickResultLstCon" data-code="3703" data-hlytag="null" data-index="4" data-lat="37.494668" data-long="127.062583" data-name="스탈릿대치R" data-storecd="1468" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="스탈릿대치R" data-store="1468" data-yn="N">스탈릿대치R  </strong> <p class="result_details">서울특별시 강남구 남부순환로 2947 (대치동)<br/>1522-3232</p> <i class="pin_reserve">리저브 매장 2번</i></li>

In [18]:
#스타벅스 매장 정보 샘플 확인
sb_store = sb_all[1]
name = sb_store.select('strong')[0].text.strip()
lat = sb_store['data-lat'].strip()
long = sb_store['data-long'].strip()
addr= str(sb_store.select('p.result_details')[0])
store_type = sb_store.select('i')[0]['class'][0][4:]
address = addr.split('<br/>')[0].split('>')[1]
tel = str(sb_store.select('p.result_details')[0])
phone = tel.split('<br/>')[1].split('<')[0]

In [20]:
print(name)         # 매장명
print(lat)          # 위도
print(long)          # 경도
print(store_type)   # 매장 타입
print(address)      # 주소
print(phone )          # 전화번호

논현역사거리
37.510178
127.022223
general
서울특별시 강남구 강남대로 538 (논현동)
1522-3232


In [21]:
#서울시 스타벅스 매장 목록 데이터 만들기
sb_list = []
for item in sb_all:
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    long = item['data-long'].strip()
    addr= str(item.select('p.result_details')[0])
    address = addr.split('<br/>')[0].split('>')[1]
    store_type = item.select('i')[0]['class'][0][4:]
    tel = str(item.select('p.result_details')[0])
    phone = tel.split('<br/>')[1].split('<')[0]
    
    sb_list.append([name, lat, long, address, store_type, phone])

columns = ['매장명','위도', '경도','주소','매장타입','전화번호']
seoul_sb_df=pd.DataFrame(sb_list,columns=columns)
seoul_sb_df.head()

,매장명,위도,경도,주소,매장타입,전화번호
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),general,1522-3232
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),general,1522-3232
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),general,1522-3232
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),general,1522-3232
4,스탈릿대치R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),reserve,1522-3232


In [22]:
#데이터프레임의 요약 정보 확인
seoul_sb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     545 non-null    object
 1   위도      545 non-null    object
 2   경도      545 non-null    object
 3   주소      545 non-null    object
 4   매장타입    545 non-null    object
 5   전화번호    545 non-null    object
dtypes: object(6)
memory usage: 25.7+ KB


In [23]:
#엑셀로 저장
seoul_sb_df.to_excel('files/seoul_sb_list.xlsx',index=False)

### 서울열린데이터광장의 OPEN API를 활용한 공공데이터 수집

In [24]:
#라이브러리 임포트
import requests
import pandas as pd
 
#자주 사용하는 변수를 미리 선언하고 인증키 저장
SEOUL_API_AUTH_KEY = '7170616b656a79313830646f636d68'
 
#호출을 위한 URL 변수 선언
url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format(SEOUL_API_AUTH_KEY)
print(url)

http://openapi.seoul.go.kr:8088/7170616b656a79313830646f636d68/json/SdeTlSccoSigW/1/25


In [25]:
#서울열린데이터광장 OPEN API 호출
result_dict = requests.get(url).json()
result_dict

{'SdeTlSccoSigW': {'list_total_count': 25,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'OBJECTID': 1.0,
    'SIG_CD': '11320',
    'SIG_KOR_NM': '도봉구',
    'SIG_ENG_NM': 'Dobong-gu',
    'ESRI_PK': 0.0,
    'LAT': '37.6658609',
    'LNG': '127.0317674'},
   {'OBJECTID': 2.0,
    'SIG_CD': '11380',
    'SIG_KOR_NM': '은평구',
    'SIG_ENG_NM': 'Eunpyeong-gu',
    'ESRI_PK': 1.0,
    'LAT': '37.6176125',
    'LNG': '126.9227004'},
   {'OBJECTID': 3.0,
    'SIG_CD': '11230',
    'SIG_KOR_NM': '동대문구',
    'SIG_ENG_NM': 'Dongdaemun-gu',
    'ESRI_PK': 2.0,
    'LAT': '37.5838012',
    'LNG': '127.0507003'},
   {'OBJECTID': 4.0,
    'SIG_CD': '11590',
    'SIG_KOR_NM': '동작구',
    'SIG_ENG_NM': 'Dongjak-gu',
    'ESRI_PK': 3.0,
    'LAT': '37.4965037',
    'LNG': '126.9443073'},
   {'OBJECTID': 5.0,
    'SIG_CD': '11545',
    'SIG_KOR_NM': '금천구',
    'SIG_ENG_NM': 'Geumcheon-gu',
    'ESRI_PK': 4.0,
    'LAT': '37.4600969',
    'LNG': '126.9001546'},
   {'OBJECTID': 6.0

In [26]:
#시군구 목록 결과 딕셔너리 데이터의 값 가져오기
result_data = result_dict['SdeTlSccoSigW']
 
print(result_data['list_total_count'])
print(result_data['RESULT'])
print(result_data['row'][0])

25
{'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}
{'OBJECTID': 1.0, 'SIG_CD': '11320', 'SIG_KOR_NM': '도봉구', 'SIG_ENG_NM': 'Dobong-gu', 'ESRI_PK': 0.0, 'LAT': '37.6658609', 'LNG': '127.0317674'}


In [30]:
data_list = result_data['row']
sample_df = pd.DataFrame(data_list)
sample_df.head()

,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546


In [28]:
# 서울열린데이터광장 API 호출 키
SEOUL_API_AUTH_KEY = '7170616b656a79313830646f636d68'
# 서울열린데이터광장 OPEN API호출 공통 함수
def seoul_open_api_data(url, service):
    data_list = None
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_list = result_data['row']
    except:
        pass
    return data_list
 
#함수를 이용해 서울시 행정구역 시군구 OPEN API 호출하기
sgg_url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format(SEOUL_API_AUTH_KEY)
sgg_data_list = seoul_open_api_data(url, 'SdeTlSccoSigW')
sgg_data_list[0]

{'OBJECTID': 1.0,
 'SIG_CD': '11320',
 'SIG_KOR_NM': '도봉구',
 'SIG_ENG_NM': 'Dobong-gu',
 'ESRI_PK': 0.0,
 'LAT': '37.6658609',
 'LNG': '127.0317674'}

In [29]:
columns = ['SIG_CD','SIG_KOR_NM','LAT','LNG']
sgg_df = pd.DataFrame(data=sgg_data_list, columns=columns)
sgg_df.head()

,SIG_CD,SIG_KOR_NM,LAT,LNG
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [31]:
sgg_df.columns=['시군구코드','시군구명','위도','경도']
sgg_df.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [32]:
sgg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구코드   25 non-null     object
 1   시군구명    25 non-null     object
 2   위도      25 non-null     object
 3   경도      25 non-null     object
dtypes: object(4)
memory usage: 928.0+ bytes


In [33]:
sgg_df.to_excel('files/seoul_sgg_list.xlsx', index=False)

In [37]:
sgg_pop_df = pd.read_csv('data3/report_인구.txt', sep='\t', encoding='utf-8', skiprows=[1,2])
sgg_pop_df.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,2020,합계,"4,417,954","9,911,088","4,816,522","5,094,566","9,668,465","4,701,723","4,966,742","242,623","114,799","127,824",2.19,"1,568,331"
1,2020,종로구,"75,003","158,996","76,876","82,120","149,384","72,635","76,749","9,612","4,241","5,371",1.99,"28,507"
2,2020,중구,"63,686","134,635","65,776","68,859","125,240","61,222","64,018","9,395","4,554","4,841",1.97,"24,495"
3,2020,용산구,"113,093","244,645","118,835","125,810","230,040","110,722","119,318","14,605","8,113","6,492",2.03,"40,247"
4,2020,성동구,"135,870","300,505","146,424","154,081","293,556","143,387","150,169","6,949","3,037","3,912",2.16,"45,603"


In [38]:
condition=sgg_pop_df['자치구'] != '합계'
sgg_pop_df_selected = sgg_pop_df[condition]
sgg_pop_df_selected.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
1,2020,종로구,"75,003","158,996","76,876","82,120","149,384","72,635","76,749","9,612","4,241","5,371",1.99,"28,507"
2,2020,중구,"63,686","134,635","65,776","68,859","125,240","61,222","64,018","9,395","4,554","4,841",1.97,"24,495"
3,2020,용산구,"113,093","244,645","118,835","125,810","230,040","110,722","119,318","14,605","8,113","6,492",2.03,"40,247"
4,2020,성동구,"135,870","300,505","146,424","154,081","293,556","143,387","150,169","6,949","3,037","3,912",2.16,"45,603"
5,2020,광진구,"167,427","360,109","173,000","187,109","346,682","167,310","179,372","13,427","5,690","7,737",2.07,"50,560"


In [39]:
sgg_pop_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   기간        25 non-null     int64  
 1   자치구       25 non-null     object 
 2   세대        25 non-null     object 
 3   인구        25 non-null     object 
 4   인구.1      25 non-null     object 
 5   인구.2      25 non-null     object 
 6   인구.3      25 non-null     object 
 7   인구.4      25 non-null     object 
 8   인구.5      25 non-null     object 
 9   인구.6      25 non-null     object 
 10  인구.7      25 non-null     object 
 11  인구.8      25 non-null     object 
 12  세대당인구     25 non-null     float64
 13  65세이상고령자  25 non-null     object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.9+ KB


In [40]:
columns=['자치구','인구']
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.head()

,자치구,인구
1,종로구,"158,996"
2,중구,"134,635"
3,용산구,"244,645"
4,성동구,"300,505"
5,광진구,"360,109"


In [41]:
sgg_pop_df_final.columns = ['시군구명','주민등록인구']
sgg_pop_df_final.head()

,시군구명,주민등록인구
1,종로구,"158,996"
2,중구,"134,635"
3,용산구,"244,645"
4,성동구,"300,505"
5,광진구,"360,109"


In [42]:
#엑셀로 저장
sgg_pop_df_final.to_excel('files/sgg_pop.xlsx', index=False)

In [44]:
sgg_biz_df = pd.read_csv('data3/report_사업체.txt', sep='\t', encoding='utf-8', skiprows=[1,2])
sgg_biz_df.head()

,기간,자치구,동,합계,합계.1,합계.2,합계.3,합계.4,농업 임업 및 어업,농업 임업 및 어업.1,...,공공행정 국방 및 사회보장 행정,공공행정 국방 및 사회보장 행정.1,교육 서비스업,교육 서비스업.1,보건업 및 사회복지 서비스업,보건업 및 사회복지 서비스업.1,예술 스포츠 및 여가관련 서비스업,예술 스포츠 및 여가관련 서비스업.1,협회 및 단체 수리 및 기타 개인 서비스업,협회 및 단체 수리 및 기타 개인 서비스업.1
0,2019,합계,합계,"823,624","281,318","5,226,997","2,885,614","2,341,383",24,426,...,"1,296","141,582","35,761","351,320","29,168","400,143","23,527","92,752","69,345","189,557"
1,2019,종로구,소계,"39,679","13,356","260,446","146,392","114,054",3,18,...,94,"15,109",829,"13,474",750,"17,366",899,"6,034","2,439","7,649"
2,2019,종로구,사직동,"3,574","1,203","47,522","25,154","22,368",1,12,...,23,"6,088",83,786,106,"1,096",91,"1,653",253,906
3,2019,종로구,삼청동,732,327,"4,601","2,271","2,330",-,-,...,4,"1,291",17,297,7,30,19,185,25,194
4,2019,종로구,부암동,599,255,"3,454","1,764","1,690",-,-,...,3,60,38,"1,190",19,183,18,49,82,325


In [47]:
condition = sgg_biz_df['동'] == '소계'
sgg_biz_df_selected = sgg_biz_df[condition]
sgg_biz_df_selected.head()

,기간,자치구,동,합계,합계.1,합계.2,합계.3,합계.4,농업 임업 및 어업,농업 임업 및 어업.1,...,공공행정 국방 및 사회보장 행정,공공행정 국방 및 사회보장 행정.1,교육 서비스업,교육 서비스업.1,보건업 및 사회복지 서비스업,보건업 및 사회복지 서비스업.1,예술 스포츠 및 여가관련 서비스업,예술 스포츠 및 여가관련 서비스업.1,협회 및 단체 수리 및 기타 개인 서비스업,협회 및 단체 수리 및 기타 개인 서비스업.1
1,2019,종로구,소계,"39,679","13,356","260,446","146,392","114,054",3,18,...,94,"15,109",829,"13,474",750,"17,366",899,"6,034","2,439","7,649"
19,2019,중구,소계,"60,127","22,451","392,568","213,484","179,084",-,-,...,65,"20,287",502,"8,598",760,"10,342",558,"3,637","2,112","8,090"
35,2019,용산구,소계,"20,254","7,074","135,240","74,570","60,670",-,-,...,47,"3,101",537,"6,973",562,"7,147",401,"2,623","1,591","5,494"
52,2019,성동구,소계,"28,343","8,563","178,243","102,769","75,474",2,5,...,45,"4,127",995,"11,678",754,"9,255",592,"2,113","2,079","7,787"
70,2019,광진구,소계,"24,445","8,921","126,305","67,899","58,406",-,-,...,36,"2,678","1,209","14,184",916,"11,551","1,052","4,209","2,552","5,828"


In [49]:
columns=['자치구','합계', '합계.2']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns=['시군구명','사업체수','종사자수']
sgg_biz_df_final.head()

,시군구명,사업체수,종사자수
1,종로구,"39,679","260,446"
19,중구,"60,127","392,568"
35,용산구,"20,254","135,240"
52,성동구,"28,343","178,243"
70,광진구,"24,445","126,305"


In [50]:
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop=True)
sgg_biz_df_final.head()

,시군구명,사업체수,종사자수
0,종로구,"39,679","260,446"
1,중구,"60,127","392,568"
2,용산구,"20,254","135,240"
3,성동구,"28,343","178,243"
4,광진구,"24,445","126,305"


In [51]:
#엑셀로 저장
sgg_biz_df_final.to_excel('files/sgg_biz.xlsx', index=False)